## Imports

In [3]:
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram

import numpy as np
import math
import random

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import cm

## Dataset generation

Output of the old generator
```python
[(2,
  array([45, 11, 40, 38]),
  {(0, 2): -15, (0, 3): -9, (1, 2): -9, (1, 3): -14}),
 (2,
  array([10, 37,  9, 46]),
  {(0, 2): -20, (0, 3): -15, (1, 2): -2, (1, 3): -4}),
 (2,
  array([12, 48, 35, 38]),
  {(0, 2): -3, (0, 3): -7, (1, 2): -19, (1, 3): -8}),
 (2,
  array([ 4, 42, 28, 33]),
  {(0, 2): -11, (0, 3): -10, (1, 2): -1, (1, 3): -14}),
 (2,
  array([43, 23, 23, 18]),
  {(0, 2): -3, (0, 3): -2, (1, 2): -16, (1, 3): 0})]
  ```

In [165]:
def create_savings(n_queries, n_plans_per_query):
    savings = {}
    for i in range(n_queries-1):
        for j in range(n_plans_per_query[i]):
            for a in range(i+1, n_queries,1):
                for b in range(n_plans_per_query[a]):
                    for x in n_plans_per_query[i+1:]:
                        for y in range(x):
                            if i == 0:
                                s = j
                                t = y + np.sum(n_plans_per_query[0:a])
                                savings[s, t] = random.randint(-20, 0)
                            else:
                                s = j + np.sum(n_plans_per_query[0:i])
                                t = y + np.sum(n_plans_per_query[0:a])
                                savings[s, t] = random.randint(-20, 0)

    return savings

In [143]:
def create_problems(n_problems, n_queries, n_plans_per_query, cost_min = 0, cost_max = 50, savings_min = -20, savings_max = 0):
    problems = []
    for i in range(n_problems):
        problems.append((n_plans_per_query, np.random.randint(cost_min, cost_max, np.sum(n_plans_per_query)), 
            create_savings(n_queries, n_plans_per_query)))
    return problems

Problems are generated, but only work, for now, in double combinations -> no three way savings!

In [170]:
problems = create_problems(1, 3, [2,2,3])
problems

[([2, 2, 3],
  array([38, 40, 20, 11, 40, 35, 19]),
  {(0, 2): -2,
   (0, 3): -17,
   (0, 4): -2,
   (0, 5): -15,
   (0, 6): -11,
   (1, 2): -3,
   (1, 3): -6,
   (1, 4): -14,
   (1, 5): -10,
   (1, 6): -15,
   (2, 4): -5,
   (2, 5): 0,
   (2, 6): -4,
   (3, 4): -9,
   (3, 5): -17,
   (3, 6): -8})]

We now generate the combinational bitstrings that are possible

In [168]:
n_qubits = np.sum(problems[0][0])
binary_string = []
for i, v in enumerate(problems[0][0]):
    if i == 0:
        for j in range(v):
            binary_string.append('0'*j + '1' + '0'*(v-j-1))
    else:
        copy = []
        for x in binary_string:
            for j in range(v):
                copy.append(x + '0'*j + '1' + '0'*(v-j-1))
        binary_string = copy


print(binary_string)


['1010100', '1010010', '1010001', '1001100', '1001010', '1001001', '0110100', '0110010', '0110001', '0101100', '0101010', '0101001']


Now we generate the circuits...

In [14]:
from qiskit import *
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

import numpy as np
import math

In [169]:
problems = create_problems(1, 3, [2,3,2])
problems

[([2, 3, 2],
  array([37,  1, 29, 28, 15, 25,  4]),
  {(0, 2): -9,
   (0, 3): -12,
   (0, 4): -13,
   (0, 5): -11,
   (0, 6): -8,
   (0, 7): -9,
   (1, 2): -16,
   (1, 3): -10,
   (1, 4): -12,
   (1, 5): -18,
   (1, 6): -12,
   (1, 7): -13,
   (2, 5): -3,
   (2, 6): -17,
   (3, 5): -7,
   (3, 6): -8,
   (4, 5): -12,
   (4, 6): -20})]

In [167]:
circuit = QuantumCircuit(np.sum(problems[0][0]))
circuit.h(range(circuit.width()))
for i, v in enumerate(problems[0][1]):
    circuit.ry(v, i)
circuit.barrier()

prev_i = 0
for i, v in problems[0][2]:
    if prev_i != i:
        circuit.barrier()
    circuit.crz(problems[0][2][i, v], i, v)
    prev_i = i
circuit.barrier()

circuit.draw()

┌───┐┌────────┐ ░                                                       ░ »
q_0: ┤ H ├┤ Ry(31) ├─░──────■──────────■─────────■─────────■──────────■──────░─»
     ├───┤├────────┤ ░      │          │         │         │          │      ░ »
q_1: ┤ H ├┤ Ry(12) ├─░──────┼──────────┼─────────┼─────────┼──────────┼──────░─»
     ├───┤├────────┤ ░ ┌────┴────┐     │         │         │          │      ░ »
q_2: ┤ H ├┤ Ry(18) ├─░─┤ Rz(-13) ├─────┼─────────┼─────────┼──────────┼──────░─»
     ├───┤├────────┤ ░ └─────────┘┌────┴────┐    │         │          │      ░ »
q_3: ┤ H ├┤ Ry(38) ├─░────────────┤ Rz(-16) ├────┼─────────┼──────────┼──────░─»
     ├───┤├────────┤ ░            └─────────┘┌───┴────┐    │          │      ░ »
q_4: ┤ H ├┤ Ry(15) ├─░───────────────────────┤ Rz(-4) ├────┼──────────┼──────░─»
     ├───┤├────────┤ ░                       └────────┘┌───┴────┐     │      ░ »
q_5: ┤ H ├┤ Ry(30) ├─░─────────────────────────────────┤ Rz(-1) ├─────┼──────░─»
     ├───┤├───────┬┘ ░                                 └────────┘┌────┴────┐ ░ »
q_6: ┤ H ├┤ Ry(6) ├──░───────────────────────────────────────────┤ Rz(-16) ├─░─»
     └───┘└───────┘  ░                                           └─────────┘ ░ »
«                                                        ░                     »
«q_0: ───────────────────────────────────────────────────░─────────────────────»
«                                                        ░                     »
«q_1: ────■─────────■─────────■─────────■─────────■──────░─────────────────────»
«     ┌───┴────┐    │         │         │         │      ░                     »
«q_2: ┤ Rz(-2) ├────┼─────────┼─────────┼─────────┼──────░─────■─────────■─────»
«     └────────┘┌───┴───┐     │         │         │      ░     │         │     »
«q_3: ──────────┤ Rz(0) ├─────┼─────────┼─────────┼──────░─────┼─────────┼─────»
«               └───────┘┌────┴────┐    │         │      ░ ┌───┴────┐    │     »
«q_4: ───────────────────┤ Rz(-10) ├────┼─────────┼──────░─┤ Rz(-2) ├────┼─────»
«                        └─────────┘┌───┴───┐     │      ░ └────────┘┌───┴────┐»
«q_5: ──────────────────────────────┤ Rz(0) ├─────┼──────░───────────┤ Rz(-9) ├»
«                                   └───────┘┌────┴────┐ ░           └────────┘»
«q_6: ───────────────────────────────────────┤ Rz(-20) ├─░─────────────────────»
«                                            └─────────┘ ░                     »
«                ░                                ░ 
«q_0: ───────────░────────────────────────────────░─
«                ░                                ░ 
«q_1: ───────────░────────────────────────────────░─
«                ░                                ░ 
«q_2: ────■──────░────────────────────────────────░─
«         │      ░                                ░ 
«q_3: ────┼──────░─────■─────────■─────────■──────░─
«         │      ░ ┌───┴────┐    │         │      ░ 
«q_4: ────┼──────░─┤ Rz(-9) ├────┼─────────┼──────░─
«         │      ░ └────────┘┌───┴────┐    │      ░ 
«q_5: ────┼──────░───────────┤ Rz(-3) ├────┼──────░─
«     ┌───┴────┐ ░           └────────┘┌───┴────┐ ░ 
«q_6: ┤ Rz(-9) ├─░─────────────────────┤ Rz(-9) ├─░─
«     └────────┘ ░                     └────────┘ ░